In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
X_train = np.load('data\X_train.npy')
y = np.load('data\y.npy')
X_test = np.load('data\X_test.npy')

In [3]:
print(X_train.shape)
print(y.shape)
print(X_test.shape)

(42000, 28, 28, 1)
(42000, 10)
(28000, 28, 28, 1)


In [4]:
# Construct data generator which applies random transformations to the images to make our data more robust

datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,)  # randomly shift images vertically (fraction of total height)


datagen.fit(X_train)

In [5]:
def get_LeNet_model(batch_size = 86, epochs=10):
    model = tf.keras.models.Sequential([
        # Convolutional layers
        tf.keras.layers.Conv2D(6, (5,5), activation='relu', input_shape=(28,28,1), padding='same', strides=(1,1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(16, (5,5), activation='relu', padding='valid', strides=(1,1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(120, (5,5), activation='relu', padding='valid', strides=(1,1)),
        
        # Flatten out
        tf.keras.layers.Flatten(),
        
        # Dense layers
        tf.keras.layers.Dense(84, activation='relu'),
        
        # Output layer
        tf.keras.layers.Dense(10, activation='softmax')
        
    ])
    
    # Compile
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=['accuracy'])
    
    # Fit the model
    history = model.fit_generator(datagen.flow(X_train,y, batch_size=batch_size),
                                  epochs = epochs, 
                                  verbose = 2, 
                                  steps_per_epoch=X_train.shape[0] // batch_size)

    return model

In [7]:
if os.path.isfile('models/LeNet.h5'):
    LeNet_model = tf.keras.models.load_model('models/LeNet.h5')
else:
    LeNet_model = get_LeNet_model()
    tf.keras.models.save_model('models/LeNet.h5')

In [8]:
def get_Dropout_model(batch_size = 86, epochs=30):
    model = tf.keras.models.Sequential([
        # Convolutional layers
        tf.keras.layers.Conv2D(32, (5,5), activation='relu', input_shape=(28,28,1), padding='same', strides=(1,1)),
        tf.keras.layers.Conv2D(32, (5,5), activation='relu', padding='same', strides=(1,1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.25),
        
        tf.keras.layers.Conv2D(64, (5,5), activation='relu', padding='same', strides=(1,1)),
        tf.keras.layers.Conv2D(64, (5,5), activation='relu', padding='same', strides=(1,1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.25),
        
        # Flatten out
        tf.keras.layers.Flatten(),
        
        # Dense layers
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        
        # Output layer
        tf.keras.layers.Dense(10, activation='softmax')
        
    ])
    
    # Compile
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=['accuracy'])
    
    # Fit the model
    history = model.fit_generator(datagen.flow(X_train,y, batch_size=batch_size),
                                  epochs = epochs, 
                                  verbose = 2, 
                                  steps_per_epoch=X_train.shape[0] // batch_size)

    return model

In [9]:
if os.path.isfile('models/Dropout.h5'):
    Dropout_model = tf.keras.models.load_model('models/Dropout.h5')
else:
    Dropout_model = get_Dropout_model()
    tf.keras.models.save_model(Dropout_model, 'models/Dropout.h5')

In [10]:
submissions = []

for model in [LeNet_model, Dropout_model]:
    # predict results
    results = model.predict(X_test)

    # select the indix with the maximum probability
    results = np.argmax(results,axis = 1)

    results = pd.Series(results,name="Label")
    
    submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
    
    submissions.append(submission)

In [11]:
submissions[0].to_csv("submissions/LeNet.csv",index=False)
submissions[1].to_csv("submissions/Dropout.csv",index=False)

In [13]:
# 0.98685 test accuracy with LeNet style model
# 0.99400 test accuracy with dropout model